In [1]:
import numpy as np
import os 
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F


In [4]:
import os
base_dir = "../models"  # Kaggle環境における入力データのベースパス
print(os.listdir(base_dir))


['.ipynb_checkpoints', 'CurveFault_A', 'CurveFault_B', 'CurveVel_A', 'CurveVel_B', 'FlatFault_A', 'FlatFault_B', 'FlatVel_A', 'FlatVel_B', 'Style_A', 'Style_B']


In [5]:
for root, dirs, files in os.walk(data_dir):
    # ルートディレクトリ名をインデント付きで表示
    depth = root.replace(data_dir, "").count(os.sep)
    indent = "  " * depth
    print(f"{indent}{os.path.basename(root) or data_dir}:")
    # すべての .npy ファイルを表示
    for f in files:
        if f.endswith(".pth"):
            print(f"{indent}  └ {f}")

models:
  .ipynb_checkpoints:
  CurveFault_A:
    └ curvefault_a_l1_480.pth
    └ curvefault_a_l2_480.pth
  CurveFault_B:
    └ curvefault_b_l1_480.pth
    └ curvefault_b_l2_480.pth
  CurveVel_A:
    └ curvevel_a_l1_480.pth
    └ curvevel_a_l2_480.pth
  CurveVel_B:
    └ curvevel_b_l1_480.pth
    └ curvevel_b_l2_480.pth
  FlatFault_A:
    └ flatfault_a_l1_480.pth
    └ flatfault_a_l2_480.pth
  FlatFault_B:
    └ flatfault_b_l1_480.pth
    └ flatfault_b_l2_480.pth
  FlatVel_A:
    └ flatvel_a_l1_480.pth
    └ flatvel_a_l2_480.pth
  FlatVel_B:
    └ flatvel_b_l1_480.pth
    └ flatvel_b_l2_480.pth
  Style_A:
    └ style_a_l1_480.pth
    └ style_a_l2_480.pth
  Style_B:
    └ style_b_l1_480.pth
    └ style_b_l2_480.pth


In [12]:
import torch
import torch.nn as nn

class ConvBlock(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size=3, stride=1, padding=1, norm=nn.BatchNorm2d, activation=nn.ReLU):
        super().__init__()
        self.layers = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size, stride, padding),
            norm(out_channels),
            activation()
        )
    def forward(self, x):
        return self.layers(x)

class DeconvBlock(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size=3, stride=1, padding=1, output_padding=0, norm=nn.BatchNorm2d, activation=nn.ReLU):
        super().__init__()
        self.layers = nn.Sequential(
            nn.ConvTranspose2d(in_channels, out_channels, kernel_size, stride, padding, output_padding),
            norm(out_channels),
            activation()
        )
    def forward(self, x):
        return self.layers(x)


# VelocityGAN

In [13]:
import torch.nn as nn

class VelocityGAN(nn.Module):
    def __init__(self, in_channels=1, out_channels=1, base_c=32, norm=nn.BatchNorm2d, activation=nn.ReLU):
        super().__init__()
        # Encoder
        self.convblock1 = ConvBlock(in_channels, base_c, norm=norm, activation=activation)
        self.convblock2_1 = ConvBlock(base_c, base_c*2, stride=2, norm=norm, activation=activation)
        self.convblock2_2 = ConvBlock(base_c*2, base_c*2, norm=norm, activation=activation)
        self.convblock3_1 = ConvBlock(base_c*2, base_c*4, stride=2, norm=norm, activation=activation)
        self.convblock3_2 = ConvBlock(base_c*4, base_c*4, norm=norm, activation=activation)
        self.convblock4_1 = ConvBlock(base_c*4, base_c*8, stride=2, norm=norm, activation=activation)
        self.convblock4_2 = ConvBlock(base_c*8, base_c*8, norm=norm, activation=activation)
        self.convblock5_1 = ConvBlock(base_c*8, base_c*8, stride=2, norm=norm, activation=activation)
        self.convblock5_2 = ConvBlock(base_c*8, base_c*8, norm=norm, activation=activation)
        self.convblock6_1 = ConvBlock(base_c*8, base_c*8, stride=2, norm=norm, activation=activation)
        self.convblock6_2 = ConvBlock(base_c*8, base_c*8, norm=norm, activation=activation)
        self.convblock7_1 = ConvBlock(base_c*8, base_c*8, stride=2, norm=norm, activation=activation)
        self.convblock7_2 = ConvBlock(base_c*8, base_c*8, norm=norm, activation=activation)
        self.convblock8 = ConvBlock(base_c*8, base_c*8, norm=norm, activation=activation)

        # Decoder
        self.deconv1_1 = DeconvBlock(base_c*8, base_c*8, stride=2, output_padding=1, norm=norm, activation=activation)
        self.deconv1_2 = DeconvBlock(base_c*16, base_c*8, norm=norm, activation=activation)
        self.deconv2_1 = DeconvBlock(base_c*8, base_c*8, stride=2, output_padding=1, norm=norm, activation=activation)
        self.deconv2_2 = DeconvBlock(base_c*16, base_c*8, norm=norm, activation=activation)
        self.deconv3_1 = DeconvBlock(base_c*8, base_c*8, stride=2, output_padding=1, norm=norm, activation=activation)
        self.deconv3_2 = DeconvBlock(base_c*16, base_c*8, norm=norm, activation=activation)
        self.deconv4_1 = DeconvBlock(base_c*8, base_c*8, stride=2, output_padding=1, norm=norm, activation=activation)
        self.deconv4_2 = DeconvBlock(base_c*16, base_c*4, norm=norm, activation=activation)
        self.deconv5_1 = DeconvBlock(base_c*4, base_c*4, stride=2, output_padding=1, norm=norm, activation=activation)
        self.deconv5_2 = DeconvBlock(base_c*8, base_c*2, norm=norm, activation=activation)
        self.deconv6 = DeconvBlock(base_c*2, base_c, stride=2, output_padding=1, norm=norm, activation=activation)

        self.final = nn.Conv2d(base_c, out_channels, 3, 1, 1)

    def forward(self, x):
        # Encoder
        x1 = self.convblock1(x)
        x2 = self.convblock2_1(x1)
        x2 = self.convblock2_2(x2)
        x3 = self.convblock3_1(x2)
        x3 = self.convblock3_2(x3)
        x4 = self.convblock4_1(x3)
        x4 = self.convblock4_2(x4)
        x5 = self.convblock5_1(x4)
        x5 = self.convblock5_2(x5)
        x6 = self.convblock6_1(x5)
        x6 = self.convblock6_2(x6)
        x7 = self.convblock7_1(x6)
        x7 = self.convblock7_2(x7)
        x8 = self.convblock8(x7)
        # Decoder
        d1 = self.deconv1_1(x8)
        d1 = torch.cat([d1, x7], dim=1)
        d1 = self.deconv1_2(d1)
        d2 = self.deconv2_1(d1)
        d2 = torch.cat([d2, x6], dim=1)
        d2 = self.deconv2_2(d2)
        d3 = self.deconv3_1(d2)
        d3 = torch.cat([d3, x5], dim=1)
        d3 = self.deconv3_2(d3)
        d4 = self.deconv4_1(d3)
        d4 = torch.cat([d4, x4], dim=1)
        d4 = self.deconv4_2(d4)
        d5 = self.deconv5_1(d4)
        d5 = torch.cat([d5, x3], dim=1)
        d5 = self.deconv5_2(d5)
        d6 = self.deconv6(d5)
        d6 = torch.cat([d6, x2], dim=1)
        out = self.final(d6)
        return out


In [14]:
import os
import torch

def collect_model_paths(root):
    model_files = []
    for subdir, _, files in os.walk(root):
        for f in files:
            if f.endswith('.pth'):
                model_files.append(os.path.join(subdir, f))
    return sorted(model_files)

def build_ensemble(model_paths, device='cuda'):
    base_model = VelocityGAN().to(device)
    state_dicts = []
    for path in model_paths:
        sd = torch.load(path, map_location=device, weights_only=False)
        # --- state_dictの在りかを自動判定 ---
        if 'model' in sd:
            # OpenFWI/VelocityGAN公式checkpoint
            state_dicts.append(sd['model'])
        elif 'state_dict' in sd:
            state_dicts.append(sd['state_dict'])
        elif 'model_state_dict' in sd:
            state_dicts.append(sd['model_state_dict'])
        else:
            # すでにstate_dictそのもの
            state_dicts.append(sd)
    # --- 平均化 ---
    avg_state = {}
    keys = state_dicts[0].keys()
    for key in keys:
        if isinstance(state_dicts[0][key], torch.Tensor):
            stacked = torch.stack([sd[key].float() for sd in state_dicts], dim=0)
            avg_state[key] = torch.mean(stacked, dim=0)
        else:
            avg_state[key] = state_dicts[0][key]
    # --- ロード ---
    base_model.load_state_dict(avg_state)
    return base_model




# 使用例
device = 'cuda' if torch.cuda.is_available() else 'cpu'
dir_root = "../models"
paths = collect_model_paths(dir_root)
print(f"Found {len(paths)} model files to ensemble.")




Found 20 model files to ensemble.


In [15]:
dir_root = "../models"

# ファイル探索で確認
import os
paths = []
for subdir, _, files in os.walk(dir_root):
    for f in files:
        if f.endswith('.pth'):
            paths.append(os.path.join(subdir, f))
print("Found .pth files:")
for p in paths:
    print(p)
print(f"Total: {len(paths)}")


Found .pth files:
../models/CurveFault_A/curvefault_a_l1_480.pth
../models/CurveFault_A/curvefault_a_l2_480.pth
../models/CurveFault_B/curvefault_b_l1_480.pth
../models/CurveFault_B/curvefault_b_l2_480.pth
../models/CurveVel_A/curvevel_a_l1_480.pth
../models/CurveVel_A/curvevel_a_l2_480.pth
../models/CurveVel_B/curvevel_b_l1_480.pth
../models/CurveVel_B/curvevel_b_l2_480.pth
../models/FlatFault_A/flatfault_a_l1_480.pth
../models/FlatFault_A/flatfault_a_l2_480.pth
../models/FlatFault_B/flatfault_b_l1_480.pth
../models/FlatFault_B/flatfault_b_l2_480.pth
../models/FlatVel_A/flatvel_a_l1_480.pth
../models/FlatVel_A/flatvel_a_l2_480.pth
../models/FlatVel_B/flatvel_b_l1_480.pth
../models/FlatVel_B/flatvel_b_l2_480.pth
../models/Style_A/style_a_l1_480.pth
../models/Style_A/style_a_l2_480.pth
../models/Style_B/style_b_l1_480.pth
../models/Style_B/style_b_l2_480.pth
Total: 20


In [16]:
ensemble_model = build_ensemble(paths, device)
save_path = os.path.join(dir_root, 'velocitygan_ensemble.pth')
torch.save(ensemble_model.state_dict(), save_path)
print(f"Ensembled model saved to: {save_path}")


RuntimeError: Error(s) in loading state_dict for VelocityGAN:
	Missing key(s) in state_dict: "final.weight", "final.bias". 
	size mismatch for convblock1.layers.0.weight: copying a param with shape torch.Size([32, 5, 7, 1]) from checkpoint, the shape in current model is torch.Size([32, 1, 3, 3]).
	size mismatch for convblock2_1.layers.0.weight: copying a param with shape torch.Size([64, 32, 3, 1]) from checkpoint, the shape in current model is torch.Size([64, 32, 3, 3]).
	size mismatch for convblock2_2.layers.0.weight: copying a param with shape torch.Size([64, 64, 3, 1]) from checkpoint, the shape in current model is torch.Size([64, 64, 3, 3]).
	size mismatch for convblock3_1.layers.0.weight: copying a param with shape torch.Size([64, 64, 3, 1]) from checkpoint, the shape in current model is torch.Size([128, 64, 3, 3]).
	size mismatch for convblock3_1.layers.0.bias: copying a param with shape torch.Size([64]) from checkpoint, the shape in current model is torch.Size([128]).
	size mism